### SageMaker endpoint

- 학습이 완료된 모델을 endpoint에 배포하여 복잡한 아키텍쳐 구성 없이 쉽게 서비스를 production 에서 활용할 수 있습니다.
- SageMaker endpoint 를 위한 script를 구성하는 방법은 여러가지가 있지만 여기서는 기본적인 방법을 활용하였습니다. `code` 디렉토리의 `inference.py` 가 예시 스크립트입니다.

In [ ]:
%store -r

In [ ]:
print(prefix)
print(test_demo_files)
# print(bucket)

In [ ]:
import torch
import matplotlib.pyplot as plt

import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

In [ ]:
sess = sagemaker.Session()
role = get_execution_role()
region = sess.boto_session.region_name
bucket = sess.default_bucket()

In [ ]:

# training_job_name = "pytorch-training-2023-09-08-08-13-53-086"
training_job_name = "pytorch-training-2023-09-08-15-50-22-545"
estimator = Estimator.attach(training_job_name)

In [ ]:
print(f"- model data: {estimator.model_data}")
print(f"- training image uri : {estimator.training_image_uri()}")

### SageMaker endpoint를 위한 container 설정

- Inference 용도의 컨테이너를 직접 제작할 수도 있고 제공되는 것을 선택할 수도 있다.

In [ ]:
from sagemaker import image_uris

instance_type = "ml.m5.4xlarge"
container_image_uri = image_uris.retrieve(
    framework='pytorch',
    region=region,
    version='1.13.1',
    py_version='py39',
    # version='1.6',
    # py_version='py3',
    image_scope='inference',
    instance_type=instance_type)

print(f"inference container uri: {container_image_uri}")

### 배포 진행

- 이 때 기존에 `model.tar.gz` 에 있는 것을 그대로 사용하게 해도 되지만, 로컬에 있는 코드를 활용하도록 하는것도 가능합니다.
- 좀 더 빠르게 디버깅 하기 위해서는 로컬 코드를 사용하도록 하는 것이 좀 더 편리합니다.

In [ ]:
## realtime endpoint
predictor = estimator.deploy(
    initial_instance_count=1,
    source_dir="code",
    entry_point="inference.py", 
    instance_type=instance_type,
    image_uri=container_image_uri,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer()
)

In [ ]:
print(predictor)

### 테스트 진행

- 성공적으로 배포가 되었다면, 테스트를 진행해 볼 수 있습니다.

In [ ]:
prefix_key = f"{prefix}/test"
file = test_demo_files[0]["image"].split("/")[-1]
print(file)
print(prefix_key)

In [ ]:
###Option 2 - use "nslicestart" and "nsliceend"

nslicestart=50
nsliceend=55
nslice=0

In [ ]:
%%time
test_demo_preds = []

###Option 1 - use totalslice
# for counter in range(totalslice): #for using totalslice

###Option 2 - use "nslicestart" and "nsliceend"
for counter in range(int(nslicestart),int(nsliceend)): #for using "slicestart" and "sliceend"
    payload={
        "bucket": bucket,
        "key": prefix_key,
        "file": file,
        "nslice": counter
    }
    response_pred = predictor.predict(payload)
    print("inference for slice", counter)
    test_demo_preds.append(response_pred)

In [ ]:
test_demo_ds = check_ds
test_demo_loader = check_loader
test_demo_data = check_data

In [ ]:
torch.tensor(test_demo_preds[nslice]["pred"]).size()

In [ ]:
image, label = (test_demo_data["image"][0][0], test_demo_data["label"][0][0])
print(f"image shape: {image.shape}, label shape: {label.shape}")

# Visualization
# plot the slice [:, :, nslice]
plt.figure("check", (18, 6))
plt.subplot(1, 3, 1)
plt.title("image")
plt.imshow(test_demo_data["image"][0, 0, :, :, nslicestart+nslice], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("label")
plt.imshow(test_demo_data["label"][0, 0, :, :, nslicestart+nslice])
plt.subplot(1, 3, 3)
plt.title("output")
plt.imshow(test_demo_preds[nslice]["pred"])

plt.show()

In [ ]:
import ast
import numpy as np

In [ ]:
%%time
slicestart=70
sliceend=75
sliceselect = f"{slicestart}:{sliceend}"

payload_multi = {
    "bucket": bucket,
    "key": prefix_key,
    "file": file,
    "nslice": sliceselect
        }

response_multi_pred = predictor.predict(payload_multi)

In [ ]:
multi_output_file = response_multi_pred["s3_path"]

download_response = sagemaker.s3.S3Downloader.read_file(multi_output_file, sagemaker_session=sess)

multi_output_json = ast.literal_eval(download_response)
multi_output = multi_output_json["pred"]

In [ ]:
output_array = np.array(multi_output)
output_array.shape

In [ ]:
image1, label1 = (test_demo_data["image"][0][0], test_demo_data["label"][0][0])
print(f"image shape: {image1.shape}, label shape: {label1.shape}\n")
for nslice in range(output_array.shape[-1]):
    # Visualization
    # plot the slice [:, :, nslice]
    print(f"Slice {slicestart+nslice}")
    plt.figure("check", (18, 6))
    plt.subplot(1, 3, 1)
    plt.title("image")
    plt.imshow(test_demo_data["image"][0, 0, :, :, slicestart+nslice], cmap="gray")
    plt.subplot(1, 3, 2)
    plt.title("label")
    plt.imshow(test_demo_data["label"][0, 0, :, :, slicestart+nslice])
    plt.subplot(1, 3, 3)
    plt.title("output")
    plt.imshow(output_array[:,:,nslice])

    plt.show()

In [ ]:
%%time

payload_all={
    "bucket": bucket,
    "key": prefix_key,
    "file": file,
    "nslice": "all"
        }

response_all_pred=predictor.predict(payload_all)

In [ ]:
all_output_file = response_all_pred["s3_path"]

download_all_response = sagemaker.s3.S3Downloader.read_file(all_output_file, sagemaker_session=sess)

all_output_json = ast.literal_eval(download_all_response)
all_output = all_output_json["pred"]

In [ ]:
output_all_array = np.array(all_output)
output_all_array.shape

In [ ]:
n_all_slice = 45

In [ ]:
image2, label2 = (test_demo_data["image"][0][0], test_demo_data["label"][0][0])
print(f"image shape: {image2.shape}, label shape: {label2.shape}")

# Visualization
# plot the slice [:, :, nslice]
plt.figure("check", (18, 6))
plt.subplot(1, 3, 1)
plt.title("image")
plt.imshow(test_demo_data["image"][0, 0, :, :, n_all_slice], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("label")
plt.imshow(test_demo_data["label"][0, 0, :, :, n_all_slice])
plt.subplot(1, 3, 3)
plt.title("output")
plt.imshow(output_all_array[:,:,n_all_slice])

plt.show()